# Projeto: Obter dados de empresas da Receita Federal, limpar e formatar os dados, realizar cruzamentos simples e gerar CSV com os dados

Bibliotecas Ultilizadas

In [7]:
require 'open-uri'
require 'zip'
require 'csv'
require 'mongo'

true

## Dowload e visualização do CSV

#### Primeiramente é feito o Dowload dos dados por meio do site da receita federal

In [12]:
url = "http://200.152.38.155/CNPJ/K3241.K03200Y0.D20312.ESTABELE.zip"
zip_file_path = "./Data/Estabelecimento01.zip"

"./Data/Estabelecimento01.zip"

In [ ]:
def dowload(url,file_path)
    File.open(file_path, "wb") do |file|
        URI.open(url).each do |zipFile|
            file.write(zipFile)
        end
    end
end
# Estabelecimento 01
dowload(url,zip_file_path)

#### Depois de ter feito o dowload é necessario extrair os arquivos que estão no formato .zip

In [ ]:
def extract_zip(zip_file_path, destination)
  
  Zip::File.open(zip_file_path) do |zip_file|
    zip_file.each do |f|
      fpath = File.join(destination, "Estabelecimento01")
      zip_file.extract(f, fpath) unless File.exist?(fpath)
    end
  end
end

extract_zip(zip_file_path,'./Data/')

#### Visualizando as Primeiras linhas do arquivo com os seus respectivos cabeçalhos

In [ ]:
headers = ["cnpj_basico","cnpj_ordem","cnpj_dv","identificador_matriz/filial","nome_fantasia","situacao_cadastral", "data_situacao_cadastral", "motivo_situacao_cadastral", "nome_cidade_no_exterior","pais", "data_de_inicio_atividade", "cnae_fiscal_principal", "cnae_fiscal_secundaria", "tipo_de_logradouro","logradouro", "numero","complemento" ,"bairro" ,"cep" ,"uf", "municipio", "ddd_1", "telefone_1", "ddd_2", "telefone_2","ddd_do_fax" , "fax","correio_eletronico", "situacao_especial", "data_da_situacao_especial"]
path = './Data/Estabelecimento01'

In [10]:
CSV.foreach(path,:headers => headers, :header_converters => :symbol,:converters => :all,:col_sep => ";").first()

#<CSV::Row cnpj_basico:36173458 cnpj_ordem:1 cnpj_dv:90 identificador_matrizfilial:1 nome_fantasia:"" situacao_cadastral:8.0 data_situacao_cadastral:20081231 motivo_situacao_cadastral:71 nome_cidade_no_exterior:"" pais:"" data_de_inicio_atividade:19900605 cnae_fiscal_principal:4713004 cnae_fiscal_secundaria:"" tipo_de_logradouro:"RUA" logradouro:"CORONEL DE MATOS" numero:45 complemento:"SALA 201" bairro:"CENTRO" cep:26262020 uf:"RJ" municipio:5869 ddd_1:"" telefone_1:"" ddd_2:"" telefone_2:"" ddd_do_fax:"" fax:"" correio_eletronico:"" situacao_especial:"" data_da_situacao_especial:"">

## Convertendo o CSV em Hashes e inserindo no MongoDb

Definindo Database e collection

In [ ]:
client = Mongo::Client.new([ '127.0.0.1:27017' ], :database => 'DadosPublicosCNPJ')
db = client.database
collection = client[:Estabelecimento01]

definindo as funções responsaveis por converter os campos de data e conversão das linhas em hashs para depois realizar a remoção dos campos invalidos

In [ ]:
campos_date = [:data_de_inicio_atividade,:data_situacao_cadastral,:data_da_situacao_especial]

def converterDatas(hash,campos)
    for campo in campos
        unless (hash[campo].nil? || hash[campo] == "0")
            hash[campo] = Date.strptime(hash[campo].to_s,'%Y%m%d')
        end
    end
end

def clean_hash_chunk(chunk,campos_date) # realiza a remoção dos campos invalidos e chama a função de converter data para a lista de linhas
    clean_hash_chunk = []
    for linha in chunk
        linha = (linha.to_hash).compact # converte a linha em hash e remove os campos invalidos
        linha.reject! { |key, value| value == "" || value == nil || (key == (:data_situacao_cadastral) && (value == "0" || value == 0))}
        converterDatas(linha,campos_date)
        clean_hash_chunk << linha # depois de limpar os campos do hash armazena em uma lista com os novos valores
    end
    return clean_hash_chunk
end

Como o arquivo é muito grande a leitura do arquivo foi feita uma linha de cada vez, escolhi realizar um armazenamento na memória por meio de pedaços(chunks) de 20000 linhas, para depois realizar a conversão em hashes e tratamento dos campos, dessa forma o processamento dos dados é mais rápido.
Logo apos esse procedimento é inserido a lista de hashes no MongoDb.

In [ ]:
chunk = []

CSV.foreach(path,"rb",liberal_parsing: true,encoding: 'iso-8859-1:utf-8',:headers => headers, :header_converters => :symbol,:col_sep => ";") do |linha|

    chunk << linha

    if chunk.size >= 20000 # armazenando a lista de hashes na memoria em chunks de 20000 linhas, para tornar o processamento mais rapido
        chunk = clean_hash_chunk(chunk,campos_date)
        collection.insert_many(chunk)
        chunk = []
    end
end
collection.insert_many(chunk)

## Descobrindo a % das Empresas Ativas

In [21]:
empresasAtivas = collection.find({"situacao_cadastral":"02"}).count()
totalEmpresas = collection.find().count()
percentual_empresas_ativas = ((empresasAtivas.to_f/totalEmpresas)*100).round(2)

puts("Empresas Ativas: #{empresasAtivas}")
puts("Total Empresas: #{totalEmpresas}")
puts("Porcentagem de ativas: #{percentual_empresas_ativas}%")


Empresas Ativas: 4022454
Total Empresas: 9079426
Porcentagem de ativas: 44.3%


## Quantas empresas do setor de restaurantes foram abertas em cada ano ?

É primeiro feita a seleção das empresas que são restaurante e em sequencia é feito um agrupamento de acordo com o ano, selecionando os campos que vão ser mantidos, e por fim o resultado vai ser uma nova collection com o nome de "RestaurantesAbertosPorAno"

In [19]:
aggregate = collection.aggregate([
    {"$match" => {"cnae_fiscal_principal" => {"$regex" => /^561/}}},
    {"$group" => {"_id" => {"$year" => "$data_de_inicio_atividade"}, "RestaurantesAbertos" => {"$sum" => 1}}},
    { "$project": {_id: 0, RestaurantesAbertos: 1, Ano: "$_id"}},
    {"$out" => "RestaurantesAbertosPorAno"}
    ])

client[:RestaurantesAbertosPorAno].find().first(4) do |documento| # apenas para visualização
    p documento
end

[{"_id"=>BSON::ObjectId('625db553f26c27611faf9fa6'), "RestaurantesAbertos"=>979, "Ano"=>1976}, {"_id"=>BSON::ObjectId('625db553f26c27611faf9fa7'), "RestaurantesAbertos"=>1, "Ano"=>1962}, {"_id"=>BSON::ObjectId('625db553f26c27611faf9fa8'), "RestaurantesAbertos"=>26309, "Ano"=>2009}, {"_id"=>BSON::ObjectId('625db553f26c27611faf9fa9'), "RestaurantesAbertos"=>5108, "Ano"=>1992}]

## Passando os resultados para um arquivo csv

Gerando CSV com as informações descobertas sobre restaurantes abertos por ano

In [ ]:
CSV.open("./Data/RestaurantesAbertosPorAno.csv",'wb') do |csv|
    csv << ["Ano","RestaurantesAbertos"]
    collection2.find().each do |document|
        csv << [document["Ano"],document["RestaurantesAbertos"]]
    end
end

Visualizando o Arquivo criado

In [40]:
CSV.foreach("./Data/RestaurantesAbertosPorAno.csv","rb") do |line|
    p line
end

["Ano", "RestaurantesAbertos"]
["1962", "1"]
["1976", "979"]
["1992", "5108"]
["2009", "26309"]
["1991", "5948"]
["1968", "312"]
["1966", "1128"]
["2011", "52075"]
["1988", "4219"]
["2003", "4476"]
["1987", "4165"]
["2007", "21559"]
["1975", "786"]
["1923", "1"]
["2019", "66186"]
["1961", "2"]
["2008", "23145"]
["2006", "13622"]
["2015", "57660"]
["2002", "4763"]
["1993", "5569"]
["1999", "5465"]
["1972", "592"]
["1971", "554"]
["1973", "701"]
["1994", "5545"]
["1989", "5116"]
["1998", "5138"]
["1985", "2951"]
["1969", "406"]
["1995", "5303"]
["1958", "2"]
["1980", "1884"]
["1964", "1"]
["1959", "2"]
["2000", "5044"]
["1983", "2385"]
["1967", "281"]
["1984", "2407"]
["2004", "4834"]
["2010", "46812"]
["2016", "54223"]
["1979", "1876"]
["1986", "4846"]
["1960", "3"]
["1978", "1717"]
["2005", "5408"]
["1996", "5035"]
["1956", "1"]
["1977", "1929"]
["2012", "52543"]
["2014", "56805"]
["1974", "681"]
["1970", "347"]
["2013", "58113"]
["2018", "60008"]
["1981", "1872"]
["1982", "2134"]
["19

671

Gerando CSV com informações da porcentagem de empresas ativas em relação ao total de empresas no arquivo "Empresa01"

In [ ]:
CSV.open("./Data/Percentual_empresas_ativas.csv",'wb') do |csv|
    csv << ["Arquivo","Total_de_Empresas","EmpresasAtivas","Percentual_de_Empresas_Ativas"]
    csv << ["Empresa01",totalEmpresas,empresasAtivas,percentual_empresas_ativas]
end

Visualizando o arquivo criado

In [29]:
CSV.foreach("./Data/Percentual_empresas_ativas.csv","rb") do |line|
    p line
end

["Arquivo", "Total_de_Empresas", "EmpresasAtivas", "Percentual_de_Empresas_Ativas"]
["Empresa01", "9079426", "4022454", "44.3"]


102